In [2]:
# Importations de packages
import numpy as np
import geopandas as gpd
import folium
import src.traitements as traitements
import pandas as pd

In [3]:
from src.config import *
from src.utils import make_path, check_dir

In [7]:
r = 25000.0 # rayon de la zone d'étude en mètres
radius_name = int(r/1000)
roi_name = METRO_NAME.lower()
# Paramétrage de la période d'étude
date_start = '2013-01-01' # date de début
date_end   = '2023-01-01' # date de fin
time_period_start = date_start[:4]
time_period_end   = date_end[:4]
years_data = int(time_period_end) - int(time_period_start)

In [8]:
name ="lyon"
year = "2013"

communes_path = os.path.join(
    communes_roi_dir.format(name, year),
    communes_roi_file_name.format(name, year)
)
communes = gpd.read_file(communes_path)

In [ ]:
old_area = bati_indus_2013_old

### Compare

In [5]:
date = '2013-01-01' # date de début
year = date[:4]
name="lyon"

### SIREN

In [118]:
siren_old = "/home/rustt/Documents/IGAST/2_PROJETS/Projet_analyse_spatiale/old_review/data/processed/SIREN/SIREN_Entrepots_2013-01-01.csv"
siren_old = pd.read_csv(siren_old, dtype={"siret":str})

In [119]:
SirenEntrepotsFolder = check_dir(processed_data_path, "SIREN")
datestr = str(date)
siren_new = pd.read_csv(make_path(siren_name.format(datestr), SirenEntrepotsFolder), dtype={"siret":str})

In [120]:
len(set(siren_old.siret).intersection(set(siren_new.siret)))

16834

In [121]:
siren_old.shape

(16834, 5)

In [122]:
siren_new.shape

(16834, 5)

### GEoSIREN

In [43]:
geosiren_old = "/home/rustt/Documents/IGAST/2_PROJETS/Projet_analyse_spatiale/old_review/data/processed/SIREN/GeoSiren_LYON_25km.csv"
geosiren_old = pd.read_csv(geosiren_old)
geosiren_old = gpd.GeoDataFrame(
    geosiren_old, geometry=gpd.points_from_xy(geosiren_old["x"], geosiren_old["y"]), crs=CRS
)

In [44]:
geosiren_old.shape

(1080181, 5)

In [42]:
from src.config import *
root_out_dir = check_dir(processed_data_path, name, "2023")
out_dir_siren = check_dir(root_out_dir, "SIREN")
geosiren_new = gpd.read_file(make_path(geosiren_name.format(name, radius_name),  out_dir_siren))

In [45]:
geosiren_new.shape

(1080181, 5)

In [48]:
len(set(geosiren_old.siret.astype(int)).intersection(set(geosiren_new.siret.astype(int))))

1080181

### Merge

In [22]:
ls /home/rustt/Documents/IGAST/2_PROJETS/Projet_analyse_spatiale/old_review/data/processed/Entrepots/csv/

Entrepots_LYON_2013-01-01_25km.csv


In [104]:
merge_old = pd.read_csv("/home/rustt/Documents/IGAST/2_PROJETS/Projet_analyse_spatiale/old_review/data/processed/Entrepots/csv/Entrepots_LYON_2013-01-01_25km.csv")
merge_old = gpd.GeoDataFrame(
    merge_old, geometry=gpd.points_from_xy(merge_old["x"], merge_old["y"]), crs=CRS
)

In [105]:
merge_new = gpd.read_file("/home/rustt/Documents/IGAST/2_PROJETS/Projet_analyse_spatiale/logistics_sprawl/data/processed/lyon/2013/Entrepots/Entrepots_lyon_2013_25km.gpkg")

In [106]:
missing_siret = set(merge_old.siret.astype(str)).difference(set(merge_new.siret.astype(str)))

In [107]:
len(missing_siret)

0

In [108]:
len(set(merge_new.siret.astype(str)).difference(set(merge_old.siret.astype(str))))

0

In [109]:
merge_new.shape

(778, 9)

### BDTOPO

In [6]:
old_bdtopo = make_path("BATI_INDUSTRIEL_{}.shp", processed_data_path, "old", "BDTOPO", "LYON", "BATI_INDUSTRIEL_{}")

In [7]:
bati_indus_2013_old = gpd.read_file(old_bdtopo.format("2013", "2013"))
bati_indus_2023_old = gpd.read_file(old_bdtopo.format("2023", "2023"))

In [39]:
new_bdtopo = make_path("bati_indus_lyon_{}.gpkg", processed_data_path, "lyon", "{}", "BDTOPO")
bati_indus_2013_new = gpd.read_file(new_bdtopo.format("2013", "2013"))
bati_indus_2023_new = gpd.read_file(new_bdtopo.format("2023", "2023"))

In [16]:
old_area = gpd.sjoin(bati_indus_2013_old, communes.dissolve(), how="inner", predicate="intersects")
new_area = gpd.sjoin(bati_indus_2013_new, communes.dissolve(), how="inner", predicate="intersects")

In [41]:
bdtopo69_2023 = gpd.read_file("/home/rustt/Documents/IGAST/2_PROJETS/Projet_analyse_spatiale/logistics_sprawl/data/raw/lyon/2023/BDTOPO/BDTOPO_3-3_TOUSTHEMES_SHP_LAMB93_D069_2023-03-15/BDTOPO/1_DONNEES_LIVRAISON_2023-03-00212/BDT_3-3_SHP_LAMB93_D069-ED2023-03-15/BATI/BATIMENT.shp")

In [17]:
old_area.shape

(27157, 14)

In [18]:
new_area.shape

(31345, 15)

In [19]:
old_area = gpd.sjoin(bati_indus_2023_old, communes.dissolve(), how="inner", predicate="intersects")
new_area = gpd.sjoin(bati_indus_2023_new, communes.dissolve(), how="inner", predicate="intersects")

In [20]:
old_area.shape

(24348, 14)

In [21]:
new_area.shape

(15966, 15)

In [46]:
bati_test_2013 = wh_new[wh_new["id"] == "BATIMENT0000000201392314"]

In [47]:
bati_test_2013

,id,NATURE,geometry
62,BATIMENT0000000201392314,Bâtiment industriel,"POLYGON Z ((849461.400 6519615.300 197.900, 84..."


In [40]:
gpd.sjoin(bati_indus_2023_new, bati_test)

,ID,NATURE_left,geometry,index_right,id,NATURE_right


In [43]:
correpondance_2023 = gpd.sjoin(bdtopo69_2023, bati_test_2013, how="inner", predicate="intersects")

In [48]:
m = correpondance_2023.to_crs(4326).explore()
bati_test_2013.to_crs(4326).explore(
    m=m,
    color='red',
    name='warehouses_t1'
)
folium.LayerControl().add_to(m)
m

In [12]:
wh_old = gpd.read_file("/home/rustt/Documents/IGAST/2_PROJETS/Projet_analyse_spatiale/old_review/data/processed/Entrepots/vecteur/Entrepots_LYON_2013_25km.gpkg")

In [15]:
wh_old.shape

(260, 2)

In [18]:
wh_old.columns

Index(['id', 'geometry'], dtype='object')

In [21]:
year = "2013"
wh_new = gpd.read_file(
    make_path(
        appariement_name.format(roi_name.upper(), year, int(r/1000)), 
        processed_data_path, name, year, "Appariement"
    )
).rename({"ID":"id"}, axis=1)

In [27]:
wh_new.dtypes

id            object
NATURE        object
geometry    geometry
dtype: object

In [32]:
wh_new.crs

<Projected CRS: EPSG:2154>
Name: RGF93 v1 / Lambert-93
Axis Info [cartesian]:
- X[east]: Easting (metre)
- Y[north]: Northing (metre)
Area of Use:
- name: France - onshore and offshore, mainland and Corsica (France métropolitaine including Corsica).
- bounds: (-9.86, 41.15, 10.38, 51.56)
Coordinate Operation:
- name: Lambert-93
- method: Lambert Conic Conformal (2SP)
Datum: Reseau Geodesique Francais 1993 v1
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [33]:
wh_old.crs

<Projected CRS: EPSG:2154>
Name: RGF93 v1 / Lambert-93
Axis Info [cartesian]:
- X[east]: Easting (metre)
- Y[north]: Northing (metre)
Area of Use:
- name: France - onshore and offshore, mainland and Corsica (France métropolitaine including Corsica).
- bounds: (-9.86, 41.15, 10.38, 51.56)
Coordinate Operation:
- name: Lambert-93
- method: Lambert Conic Conformal (2SP)
Datum: Reseau Geodesique Francais 1993 v1
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [31]:
gpd.sjoin(wh_old, wh_new, predicate="intersects")

,id_left,geometry,index_right,id_right,NATURE
1,19008,"POLYGON Z ((845251.100 6492728.300 159.400, 84...",144,BATIMENT0000000238819038,Bâtiment industriel
2,19039,"POLYGON Z ((845527.600 6493553.700 164.400, 84...",145,BATIMENT0000000238819744,Bâtiment industriel
3,19115,"POLYGON Z ((845604.400 6493735.300 171.000, 84...",54,BATIMENT0000000317254238,Bâtiment industriel
5,20229,"POLYGON Z ((849951.400 6507067.700 233.000, 84...",56,BATIMENT0000000317255348,Bâtiment industriel
6,20336,"POLYGON Z ((849591.600 6507538.000 213.700, 84...",64,BATIMENT0000000013466857,Bâtiment industriel
...,...,...,...,...,...
253,84115,"POLYGON Z ((832528.700 6520322.700 311.500, 83...",105,BATIMENT0000000235948014,Bâtiment industriel
254,84123,"POLYGON Z ((832630.000 6520246.500 309.400, 83...",106,BATIMENT0000000235948028,Bâtiment industriel
255,84403,"POLYGON Z ((839837.500 6520531.400 183.600, 83...",194,BATIMENT0000000240885161,Bâtiment industriel
257,84487,"POLYGON Z ((839957.700 6521550.200 180.900, 83...",73,BATIMENT0000000240871468,Bâtiment industriel


In [26]:
len(set(wh_new["id"]).difference(wh_old["id"]))

224

In [ ]:
set(wh_app_old.ID).difference(wh_new.ID)

### To add
* rename columns
* astype id wh
* remove dept Loire
* drop duplicates